# Comparing memory usage for 3 packages: `pytraj`, `mdtraj`, `MDAnalysis`

#### Hints: `pytraj` is 3 times less efficient in memory than `mdtraj`. `MDAnalysis` uses iterator all the time so it's very friendly with RAM

In [1]:
# require: ipython_memory_usage.py (https://github.com/ianozsvald/ipython_memory_usage) # to profile
# every line of code in ipython and notebook
# https://pypi.python.org/pypi/memory_profiler
# https://pypi.python.org/pypi/psutil

# Note: I turned off MDAnalysis since getting error about netcdf file when running this notebook several times.

In [2]:
import netCDF4 as netcdf
import ipython_memory_usage as imu
import mdtraj as md
import pytraj.io as io
from MDAnalysis import Universe
imu.start_watching_memory()

In [2] used 40.6172 MiB RAM in 0.90s, peaked 0.00 MiB above current, total RAM usage 72.96 MiB


## Loading

In [3]:
## loading to memory
topname = "../tests/data/nogit/remd/myparm.top"
fname = "../tests/data/nogit/remd/remd.000.ncdf"

In [3] used 0.0430 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 73.00 MiB


In [4]:
# load mdtraj
mtraj = md.load(fname, top="../tests/data/nogit/remd/myparm.parm7")

In [4] used 219.5156 MiB RAM in 2.09s, peaked 199.68 MiB above current, total RAM usage 292.52 MiB


In [5]:
# load MDAnalysis
u = Universe(topname, fname)

In [5] used 14.8555 MiB RAM in 1.05s, peaked 0.00 MiB above current, total RAM usage 307.37 MiB


In [6]:
# load pytraj: TrajReadOnly
trajread = io.load(fname, topname)

In [6] used 5.4648 MiB RAM in 0.31s, peaked 0.00 MiB above current, total RAM usage 312.84 MiB


In [7]:
# load pytraj: FrameArray
fa = io.load(fname, topname)[:]

# FrameArray is 3 times heavier than mtraj (from mdtraj), both trajectories are in memory. The file size is ~200 MB.
# FrameArray is too fat. :D

In [7] used 756.9922 MiB RAM in 2.64s, peaked 0.00 MiB above current, total RAM usage 1069.83 MiB


## Slicing - single index

In [8]:
# MDAnalysis
u.trajectory[100]

< Timestep 101 with unit cell dimensions [  61.17625427   61.17625427   61.17625427  109.47122192  109.47122192
  109.47122192] >

In [8] used 0.0742 MiB RAM in 0.13s, peaked 0.00 MiB above current, total RAM usage 1069.90 MiB


In [9]:
# mdtraj
mtraj[100]

<mdtraj.Trajectory with 1 frames, 17443 atoms, 5666 residues, and unitcells at 0x2aaad36718d0>

In [9] used 9.5820 MiB RAM in 0.46s, peaked 0.00 MiB above current, total RAM usage 1079.48 MiB


In [10]:
# pytraj
trajread[100]

<Frame with 17443 atoms>

In [10] used 0.5430 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 1080.03 MiB


In [11]:
# pytraj, FrameArray
fa[100]

<Frame with 17443 atoms>

In [11] used 0.1797 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 1080.21 MiB


## Slicing - a chunk

In [12]:
# MDanalysis, they use iterator. memory is very small
u.trajectory[200:500]

<generator object iterNETCDF at 0x2aaad4589b90>

In [12] used 0.0039 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 1080.21 MiB


In [13]:
# mdtraj
mtraj[200:500]

<mdtraj.Trajectory with 300 frames, 17443 atoms, 5666 residues, and unitcells at 0x2aab08407510>

In [13] used 70.4102 MiB RAM in 0.57s, peaked 0.00 MiB above current, total RAM usage 1150.62 MiB


In [14]:
# pytraj, TrajReadOnly
trajread[200:500]

<FrameArray with 300 frames, 17443 atoms/frame>
           

In [14] used 279.2930 MiB RAM in 0.89s, peaked 0.00 MiB above current, total RAM usage 1429.91 MiB


In [15]:
# pytraj, FrameArray, too much memory
fa[200:500]

<FrameArray with 300 frames, 17443 atoms/frame>
           

In [15] used 187.6641 MiB RAM in 0.56s, peaked 0.00 MiB above current, total RAM usage 1617.58 MiB


# Conclusion
* MDanalysis is the best friend of RAM
* mdtraj: 2nd
* pytraj: FrameArray is too fat (is there anyway to have both supperfast iterating + less memory?). But we have TrajReadOnly two